# Reshape data: long and wide

Converting between long format and a wide format is a common task. There's two recommended ways of doing it depending on if we want to run things on index or not. If so, it is preferable to use stack. Otherwise, melt and pivot are more suitable. 

Also, stack will likely end up with a df series, and that might not be desirable sometimes... And unstack might sometimes end with multi-index on the column side. And melt and pivot are free from these issues.

In [59]:
import pandas as pd 
import numpy as np
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado']),
                    columns=pd.Index(['one', 'two', 'three']))
stacked_ = data.stack()
stacked_ #the column will be part of the index

Ohio      one      0
          two      1
          three    2
Colorado  one      3
          two      4
          three    5
dtype: int64

In [51]:
#need some cleaning up if we want it to be a dataframe
cleanup = stacked_.to_frame().reset_index()
cleanup.columns = ['state', 'var_name', 'values']
cleanup

,state,var_name,values
0,Ohio,one,0
1,Ohio,two,1
2,Ohio,three,2
3,Colorado,one,3
4,Colorado,two,4
5,Colorado,three,5


In [60]:
#might be multi-index there in the column
unstacked = cleanup.set_index(['state', 'var_name']).unstack()
unstacked

values          
var_name    one three two
state                    
Colorado      3     5   4
Ohio          0     2   1

In [68]:
data_2 = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado']),
                    columns=pd.Index(['one', 'two', 'three']))
data_2 = data_2.reset_index()

In [81]:
melted = pd.melt(data_2, id_vars=['index'], value_vars=['one', 'two', 'three'])
melted

,index,variable,value
0,Ohio,one,0
1,Colorado,one,3
2,Ohio,two,1
3,Colorado,two,4
4,Ohio,three,2
5,Colorado,three,5


In [75]:
#the inverse operation
melted.pivot('index', 'variable', 'value')

variable,one,three,two
index,,,
Colorado,3,5,4
Ohio,0,2,1


The three args in this function is index, column and value. Note that we do not have to specify the 'value' part, but if we do not do that explicitly, usually we will end up with a multi-index column. Here's a demo.

In [76]:
melted.pivot('index', 'variable')

value          
variable   one three two
index                   
Colorado     3     5   4
Ohio         0     2   1

# example from textbook
The following is the textbook example to show this.

In [121]:
marco_data = pd.read_csv('macrodata.csv')
dt_index = pd.PeriodIndex(year=marco_data.year, quarter=marco_data.quarter)
marco_data.index = dt_index.to_timestamp('D','end')
marco_data = marco_data[['realgdp', 'infl', 'unemp']]

In [137]:
long = marco_data.stack().reset_index()#get the long format from wide one
long.columns = ['date', 'variable','value']
long.head()

,date,variable,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340


In [138]:
wide = long.pivot('date', 'variable', 'value') #get back to long format from wide
wide.head()

variable,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


# concat 

concat and merge are both ways to combine dataframes. concat is more like stacking dataframes/series together and merge is more like SQL operations.

In [139]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [148]:
pd.concat([s1, s2, s3], axis=1, sort=False)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [149]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

There's several twist in here. First of all, we can use join='inner' to only keep the indices that matched up. Hence, for the three series we mentioned before, we should get an empty df if join is set to inner since there's no index overlap.

Secondly, if one wishes to retrieve individual dfs, one way is to set a key. 

In [152]:
pd.concat([s1, s2, s3], axis=1, join='inner')

,0,1,2


In [153]:
#set key for later retrial
pd.concat([s1, s2, s3], keys=['first', 'second', 'third'])

first   a    0
        b    1
second  c    2
        d    3
        e    4
third   f    5
        g    6
dtype: int64

Sometimes there might be collision in index. If we do not care, then just use the pd.concat as it is. However, we can set verify_integrity=True, and an error will be raised if there's collision instead of letting it passing by silently.

In [157]:
s1 = pd.Series([0, 1,100], index=['a', 'b', 'c'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [160]:
try:
    pd.concat([s1, s2, s3], verify_integrity=True)
except ValueError as e:
    print(e)

Indexes have overlapping values: Index(['c'], dtype='object')


Lastly, if we do not even want to keep index, then do igore_index=True and the index infor will be discarded. Instead, integers are going to be used as the index by default.


In [162]:
s1 = pd.Series([0, 1,100], index=['a', 'b', 'c'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
pd.concat([s1, s2, s3], ignore_index=True)

0      0
1      1
2    100
3      2
4      3
5      4
6      5
7      6
dtype: int64

# merge

Additionally, it's ok to merge on multiple columns as well, and it can be done by simply passing a list of There's several different variations based on where to merge and how to merge. Basically, we can choose merge on column or merge on index, or a mixture of these two, for that matter. And on how to merge, there's outer, inner, left and right. Underneath the hood, there's also one to one, one to many and many to many merge. However, these are abstracted from us but they do have well-define behaviors. The last one, although might sounds confusing, is simply a Cartesian product. 

In [169]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})
right1 = pd.DataFrame({'value': [3.5, 7]}, index=['a', 'b'])
pd.merge(left1, right1, left_on='key', right_index=True, how='left')

,key,value_x,value_y
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


Another thing that we can do is to give name suffixes so that there's no name collision. For instance, sometimes both will have a column called name (not key), and the merged dataframe will make name_x, and name_y. We can, however, overwrite this behavior directly.

In [170]:
pd.merge(left1, right1, left_on='key', right_index=True, how='left', suffixes=['_left', ''])

,key,value_left,value
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN
